### Scraper Ministero dello Sviluppo Economico (MISE) - Consumi Petroliferi 

In [343]:
# Libraries 
import requests
import bs4
import re
import os
import urllib
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

cwd = os.getcwd()
pd.set_option('display.max_rows', None) # display all rows df 

***
![alt text](scraper-image.png)
***

In [340]:
# Get request and parse 
res = requests.get("https://dgsaie.mise.gov.it/consumi_petroliferi.php")
res.raise_for_status()
mise = bs4.BeautifulSoup(res.text, "lxml")

In [341]:
# Extracting link from bs4 parser 
links = []
pre_consuntivi = []
definitivi = []

# Taking urls as string 
for link in mise.find_all('a',text=re.compile(".xls")):
    links.append(link.get('href'))

# Choose 
for link in links:
    if "preconsuntivi" in link:
        pre_consuntivi.append(link)
    else:
        definitivi.append(link)

In [342]:
# Read online excel without download 
def excel_to_pandas(URL, local_path):
    '''Get an url of an excel file, read its contents
         and convert to pandas df '''
    resp = requests.get(URL)
    with open(local_path, 'wb') as output:
        output.write(resp.content)
    df = pd.read_excel(local_path)
    return df

In [344]:
#----------------------
# Step 1. Preconsuntivi 
#----------------------

df1 = []

for link in tqdm(pre_consuntivi):

    # Read url content 
    df = excel_to_pandas(link,cwd+"/download.xls")
    
    # Take total consumptions and header
    rows = [1,2,df[df.iloc[:,0]=="TOTALE  CONSUMI"].index.item()]
    df = df.iloc[rows]
    # Delete bimestrial data
    df = df.iloc[:,[2]]
    
    # Build df 
    df = pd.DataFrame({"Mese": df.iloc[0],
                       "Anno": df.iloc[1],
                       "Consumi": df.iloc[2]}).reset_index(drop=True)
    
    # Clean 
    df.Mese = df.Mese.str.slice(stop=3).str.lower()
    # Append
    df1.append(df)
    
df1 = pd.concat(df1,axis=0)

#-----------------
# Step 2. Defitivi
#-----------------
df2 = []

for link in tqdm(definitivi):
    
    # Read url content 
    df = excel_to_pandas(link,cwd+"/download2.xls")
    
    # Take total consumption 
    rows = [2, df[df.iloc[:,1]=="CONSUMI"].index.item()]
    df = df.iloc[rows,2:]
    # Find all strings that after a dot contains upper-case letters
    mask = df.iloc[0].str.contains('\.[A-Z]',regex=True,na=False)
    # Find all strings that contains only digits 
    mask2 = df.iloc[1].str.contains('[0-9]', regex=True, na=True)
    # Filter with boolean mask 
    df = df.loc[:,(mask==False) & (mask2==True)]
    # Take years of the time-series (set black everything that is not digit)
    anno = re.sub('\D', "", link)

    # Build df 
    df = pd.DataFrame({"Mese": df.iloc[0],
                       "Anno": anno,
                       "Consumi": df.iloc[1]}).reset_index(drop=True)
    # Clean 
    df.Mese = df.Mese.str.replace(".","").str.lower()
    # Append
    df2.append(df)
    
df2 = pd.concat(df2,axis=0)

#---------------------------------
# Step 3. Concatenate and save csv
#---------------------------------
df = df1.append(df2)
df.to_csv('consumi-storico.csv')


100%|██████████| 17/17 [00:04<00:00,  3.54it/s]


*** 